# EFD  from summit,   Zk from USDF : 20230310 

In [1]:
dx = [0.1679534614086151,
 -1.9682248830795288,
 -1.5237804651260376,
 -1.0793360471725464,
 -0.6348915100097656,
 -0.19044706225395203,
 0.2539973855018616,
 0.6984418630599976,
 1.1428862810134888,
 1.58733069896698,
 2.0317752361297607]



But which seqNum this corresponds to ? Need to find out which seqNum is which dx... Let's get them here:

In [22]:
from astropy.time import Time, TimeDelta
import pandas as pd
from lsst_efd_client import EfdClient
from astropy.io import fits
from astropy.visualization import ZScaleInterval
import matplotlib.pyplot as plt
import numpy as np 
from lsst.daf import butler as dafButler
from lsst.pipe.tasks.quickFrameMeasurement import QuickFrameMeasurementTask
from lsst.ts.wep.cwfs.Instrument import Instrument

In [23]:
efd_client = EfdClient('usdf_efd')

Get the beginning and end of observations to query for a smaller subset of EFD:

In [34]:
spans = []

day_obs = '20230310'
butler = dafButler.Butler('/sdf/data/rubin/repo/embargo/')
datasetRefs = butler.registry.queryDatasets('raw',collections='LATISS/raw/all',
                              where=f"instrument='LATISS' AND exposure.day_obs = {day_obs}").expanded()


for i, ref in enumerate(datasetRefs):
    record = ref.dataId.records["exposure"]
    exp = record.dataId['exposure']
    spans.append(record.timespan)
    
t1= min(spans)
t2 = max(spans)

end_readout = await efd_client.select_time_series("lsst.sal.ATCamera.logevent_endReadout", 
                                          '*', t1.begin.utc, t2.end.utc)


This contains all entries from that night.  Need to pull only what pertains to the CWFS doubles.

The `end_readout` contains information about when the data was done reading. Print it for the exposures of interest: 

Store only the exposure name, and the timestamp of the end readout: 

In [120]:
m = (end_readout['imageNumber'] > 31) * (end_readout['imageNumber'] < 53)

subset = end_readout[m]
intra_images = []
extra_images = []
intra_times = []
extra_times = []
intra_exptimes = []
extra_exptimes = []


intra_programs = []
extra_programs = []
for i in range(len(subset)):
    num = subset['imageNumber'][i]
    values = subset['additionalValues'][i]
    imageName = subset['imageName'][i]
    time = subset.index[i] # time of end readout 
    timestamp = ''.join(values.split(':')[1:-6])
    program = values.split(':')[-2]
    
    
    if program.startswith('INTRA_AOS_SM_offset'):
        intra_times.append(time)
        intra_images.append(imageName)
        intra_programs.append(program)
    elif program.startswith('EXTRA_AOS_SM_offset'):
        extra_times.append(time)
        extra_images.append(imageName)
        extra_programs.append(program)
    #print(num, program, time, timestamp)
    #print(end_readout[m].iloc[i][['imageNumber','additionalValues']])
    


Now pull the requisite EFD information:

In [132]:
for i in range(len(intra_times)):
    
    t1 = Time(intra_times[i]) - TimeDelta(20, format='sec') \
         - TimeDelta(15., format='sec') # this is 15 sec before the beginning of exposure 
    exp_start = Time(intra_times[i]) - TimeDelta(20, format='sec')
    exp_end = Time(intra_times[i])

    t2 = Time(extra_times[i]) - TimeDelta(2., format='sec')
    # this is 2 sec before the end of the extra-focal exposure 
    
    hexapod_vals = await efd_client.select_time_series("lsst.sal.ATHexapod.positionStatus", 
                    ["reportedPosition0", "reportedPosition1",
                     "reportedPosition2", "reportedPosition3", 
                     "reportedPosition4", "reportedPosition5"], 
                                                       t1 , t2)

    z = hexapod_vals['reportedPosition2']#.median()
    
    correction = await efd_client.select_time_series("lsst.sal.ATAOS.logevent_correctionOffsets",
                                                     ["x","y","z","u","v","w"], t1,t2)
    
    print('\n\nt1=', t1, '\nt2=', t2, intra_programs[i], 
          extra_programs[i],'\nz=', correction['z'])




t1= 2023-03-11 00:30:03.962897 
t2= 2023-03-11 00:31:04.829711 INTRA_AOS_SM_offset_x_-2.0 EXTRA_AOS_SM_offset_x_-2.0 
z= 2023-03-11 00:30:13.005943+00:00    0.828178
2023-03-11 00:30:14.406828+00:00    0.828178
2023-03-11 00:30:37.453930+00:00    0.790278
2023-03-11 00:30:43.921756+00:00   -0.810822
Name: z, dtype: float64


t1= 2023-03-11 00:31:28.815401 
t2= 2023-03-11 00:32:29.988605 INTRA_AOS_SM_offset_x_-1.5555555555555556 EXTRA_AOS_SM_offset_x_-1.5555555555555556 
z= 2023-03-11 00:31:30.274166+00:00   -0.009722
2023-03-11 00:31:40.681550+00:00    0.790278
2023-03-11 00:32:02.193185+00:00    0.790278
2023-03-11 00:32:03.208524+00:00    0.790278
2023-03-11 00:32:08.964893+00:00   -0.810821
2023-03-11 00:32:09.363572+00:00   -0.810821
Name: z, dtype: float64


t1= 2023-03-11 00:32:51.641787 
t2= 2023-03-11 00:33:52.326142 INTRA_AOS_SM_offset_x_-1.1111111111111112 EXTRA_AOS_SM_offset_x_-1.1111111111111112 
z= 2023-03-11 00:32:53.207539+00:00   -0.009721
2023-03-11 00:33:03.698161+0

Now the problem is that these do not show anyhow the `2.0` offset that should be there  for hte last pair.. Where can I get it from?  

Get the Zernikes:

In [16]:
from lsst.daf import butler as dafButler
butler = dafButler.Butler('/repo/embargo',  collections=['u/jmeyers3/ATMarch10'])

In [17]:
dataIds = list(butler.registry.queryDataIds(['exposure', 'detector'], 
                                            collections='LATISS/raw/all', 
                                            datasets='raw', 
                                            instrument="LATISS",
 where='(exposure.day_obs IN ( 20230310,20230311)) \
 AND (exposure.seq_num > 22)\
 AND (exposure.seq_num < 53)'
                                           
                                           ))


In [19]:
datasetRefs = butler.registry.queryDatasets('zernikeEstimateRaw',
                              where=f"instrument='LATISS'").expanded()

